In [1]:
import findspark
findspark.init()
findspark.find()

'H:\\SPARK'

In [2]:
from pyspark.sql import SparkSession

# Initialize SparkSession with necessary configurations
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Spark') \
    .config("spark.driver.memory", "15g") \
    .config("spark.hadoop.home.dir", "H:/HADOOP/") \
    .config("spark.hadoop.conf.dir", "H:/HADOOP/etc/hadoop/") \
    .getOrCreate()
    
import sys
sys.path.append("G:\Dissertation_Project")

# Get SparkContext from the SparkSession
sc = spark.sparkContext


In [3]:
spark

### BASE DATASET

In [4]:
base_df = spark.read.csv("../../Data/Custom_Datasets/conversation_datasets_GPT.csv", header=True, inferSchema=True)
base_df.show(10, truncate=False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|Conversation_ID|Attacker_Helper                                                                                                                                                 |Victim                                                                                                                                                                                         |Conversation_Type|
+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------

### PREPROCESSED DATASET

In [5]:
preprocessed_df = spark.read.csv("../../Data/Preprocessed_Datasets/GPT_dataset_preprocessed.csv", header=True, inferSchema=True)
preprocessed_df.show(10, truncate=False)

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Convert Conversation Columns into actual Arrays

In [6]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import ast

# UDF to convert string representation of list to actual list
def str_to_array_of_arrays(s):
    # Convert the string to a list and then wrap it inside another list
    return [ast.literal_eval(s)][0]

str_to_array_of_arrays_udf = udf(str_to_array_of_arrays, ArrayType(ArrayType(StringType())))

df = preprocessed_df.withColumn("Attacker_Helper", str_to_array_of_arrays_udf(preprocessed_df["Attacker_Helper"])).withColumn("Victim", str_to_array_of_arrays_udf(preprocessed_df["Victim"]))

df.printSchema()

root
 |-- Conversation_ID: string (nullable = true)
 |-- Attacker_Helper: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Victim: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Conversation_Type: integer (nullable = true)



### Loading the pipeline and transforming the data

In [7]:
from pyspark.ml import PipelineModel
from src.CustonTransformers import FlattenTransformer

pipeline_model_path = "./Pipelines/TF-IDF_Pipeline"

pipeline = PipelineModel.load(path=pipeline_model_path)

df_assembled = pipeline.transform(df)

### Splitting the Data

In [8]:
(train_data, test_data) = df_assembled.randomSplit([0.8, 0.2], seed=42)
train_data.show(truncate=False)

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# LOGISTIC REGRESSION

### Train the Logistic Regression Model

In [9]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(featuresCol="combined_features", labelCol="Conversation_Type", predictionCol="Prediction", maxIter=50)

evaluator = MulticlassClassificationEvaluator(labelCol='Conversation_Type', metricName="accuracy", predictionCol="Prediction")

paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [10, 100]) \
    .addGrid(lr.regParam, [0.01, 0.1, 1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()


# This is similar to cross validation but only split the set once
tvs = TrainValidationSplit(
    estimator=lr, 
    estimatorParamMaps=paramGrid, 
    evaluator=evaluator, 
    parallelism=2, 
    seed=42)


tvsModel = tvs.fit(train_data)

print(tvsModel, '\n')
print(tvsModel.explainParams(), '\n')
print('\nEvaluating with metric -> {}'.format(evaluator.getMetricName()))
print('Evaluators rating for the test set -> {}'.format(evaluator.evaluate(tvsModel.transform(test_data))))


TrainValidationSplitModel_9755a78189dc 

estimator: estimator to be cross-validated (current: LogisticRegression_8a118c9d2bec)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LogisticRegression_8a118c9d2bec', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_8a118c9d2bec', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_8a118c9d2bec', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, {Param(parent='LogisticRegression_8a118c9d2bec', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_8a118c9d2bec', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_8a118c9d2bec', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an 

In [10]:
print('Train Validation Split Model validationMetrics -> {}'.format(tvsModel.validationMetrics))
print('Best Model -> {}'.format(tvsModel.bestModel))

Train Validation Split Model validationMetrics -> [0.8571428571428571, 0.8857142857142857, 0.8285714285714286, 0.8571428571428571, 0.8285714285714286, 0.8571428571428571, 0.8285714285714286, 0.5714285714285714, 0.5714285714285714, 0.8571428571428571, 0.8857142857142857, 0.8285714285714286, 0.8571428571428571, 0.8285714285714286, 0.8571428571428571, 0.8285714285714286, 0.5714285714285714, 0.5714285714285714]
Best Model -> LogisticRegressionModel: uid=LogisticRegression_8a118c9d2bec, numClasses=2, numFeatures=36


### Testing

In [11]:
predictions = tvsModel.bestModel.transform(test_data)
predictions.select("Conversation_ID","Conversation_Type", "Prediction", "Probability").where(predictions.Prediction != predictions.Conversation_Type).show(10, truncate=False)

+---------------+-----------------+----------+----------------------------------------+
|Conversation_ID|Conversation_Type|Prediction|Probability                             |
+---------------+-----------------+----------+----------------------------------------+
|FKkPuYQFmm4wqK |0                |1.0       |[0.2962591727164577,0.7037408272835424] |
|fdVlWeFdGUkhrS |0                |1.0       |[0.4468155800141608,0.5531844199858392] |
|u6jomu1BoLjgOb |0                |1.0       |[0.22286164318924317,0.7771383568107568]|
+---------------+-----------------+----------+----------------------------------------+



In [12]:
model_path = "../Models/Trained_Models/LogisticRegression"
print(type(model_path))

<class 'str'>


In [13]:
tvsModel.write().overwrite().save(model_path)